In [7]:
import numpy as np
from pressure_vessel_edit import PressureVessel
import random 
import time
import statistics

In [8]:
simulated_index=[]
batch_start=250
filename= 'graph_w_data_'+str(batch_start)+'.csv'
rhog_safety=15096.9      # calculted as pho=1027, g=9.8, safety factor=1.5 ( from STR excel sheet)
threshold=0.05           # 5 percent ( Coefficient of variation= std/mean) 
window_len= 6            # window to check convergence

In [21]:
designset_load= np.loadtxt('./indexed_design_point.txt', delimiter = " ")
design_set=designset_load[random.sample(range(0, 24000), 6000)]
print(design_set)
np.savetxt('sampled_indexed_design_point.txt',design_set)

[[9.8180e+03 6.0000e+02 5.0000e-01 1.8000e-01]
 [1.9440e+03 4.9000e+03 1.0000e-01 2.4000e-01]
 [1.9089e+04 5.8000e+03 8.0000e-01 9.0000e-02]
 ...
 [2.2350e+04 1.9000e+03 1.0000e+00 3.0000e-01]
 [1.6164e+04 4.5000e+03 7.0000e-01 4.0000e-02]
 [1.7240e+04 1.1000e+03 8.0000e-01 4.0000e-01]]


In [9]:
designset_load= np.loadtxt('./sampled_indexed_design_point.txt', delimiter = " ")
design_set=designset_load[batch_start:,:]
print(design_set.shape)

(5750, 4)


In [10]:
vessel = PressureVessel('pv_capsule1.FCStd')
vessel.set_tensile_strength(276)
vessel.set_youngs_modulus(68900)
csvwriter=vessel.csv_open_output(filename)
vessel.print_info() 

Opening: pv_capsule1.FCStd
file name is: graph_w_data_250.csv
opening file to write
field name is: ['thickness', 'radius', 'length']
Object names: Body, Origin, X_Axis, Y_Axis, Z_Axis, XY_Plane, XZ_Plane, YZ_Plane, Sketch, Revolution, Analysis, SolverCcxTools, MaterialSolid, FEMMeshGmsh, ConstraintFixed, ConstraintPressure
Sketch parameters:
thickness = 0.01
radius = 0.242590379800142
length = 0.1
Body properties:
  body_area = 1.717916 m^2
  body_volume = 0.008587486 m^3
  body_mass = 23.186 kg
  outer_area = 0.891956 m^2
  outer_volume = 0.078289424 m^3
  inner_area = 0.825960 m^2
  inner_volume = 0.069701938 m^3
FEM parameters:
  pressure = 21.135659999999998 MPa
  mesh_length = 0.06600000000000003 m
Material parameters:
  youngs_modulus = 68900.0 MPa
  poisson_ratio = 0.33
  tensile_strength = 276.0 MPa
  density = 2700.0 kg/m^3
Mesh properties:
  node_count = 3281
  edge_count = 118
  face_count = 1088
  volume_count = 1643
FEM Results: none


In [11]:
convergence_data=[]

def listOfTuples(l1, l2):
    return list(map(lambda x, y:(x,y), l1, l2))

def capture_datumerror(crush_pressure,thickness_d,radius_d,length_d):
    try: 
     vessel.set_pressure(crush_pressure)
     vessel.set('thickness', thickness_d)
     vessel.set('radius', radius_d)
     vessel.set('length', length_d)
     return 0
    except:
     print("Something went wrong in setting value")
     return 1

def check_convergence(predictions,ml,experiment_id):
    if len(predictions)<=window_len :
        return False
    else: 
       windowed_prediction = predictions[-window_len:]
       windowed_ml = ml[-window_len:]
       print('windowed result is:',windowed_prediction)
       std_w=statistics.stdev(windowed_prediction)
       mean_w=statistics.mean(windowed_prediction)
       if (std_w/mean_w)>threshold:
          return False
       else:
          convergence_data.append(experiment_id)
          convergence_data.append(listOfTuples(windowed_prediction,windowed_ml))
          return True

In [ ]:
skipped=[]
start=time.time()
#mesh_options=[0.08,0.04,0.035,0.030,0.025,0.020,0.01,0.008,0.004,0.002,0.001]
mesh_options=np.arange(0.03,0.1,.002)[::-1]
print('mesh option is :',mesh_options)
for i in range(design_set.shape[0]):
     experiment_id=design_set[i][0]
     depth= design_set[i][1]
     length_d= design_set[i][2]
     thickness_d= design_set[i][3]
     radius_d= thickness_d+ np.random.rand()*0.4+0.1  
      
     print('---> depth is:',depth,'length is:',length_d,'thickness is:',thickness_d, 'radius is:',radius_d )
    
        
     crush_pressure= rhog_safety*depth*0.000001
   
     de=capture_datumerror(crush_pressure,thickness_d,radius_d,length_d)
     vessel.set_exp_index(experiment_id)
     predicted_vonmises=[]; mesh_lengths=[]
     if de==0:
      k=0 
      for j in range(len(mesh_options)):
        ml= mesh_options[j]
        print('ml is:',ml)
        vessel.set('mesh_length', ml)
        k=vessel.run_analysis()
        print('itr is:',i,',k is:',k, ',ml is:',ml)
        if k==2:  
            mesh_lengths.append(ml)
            predicted_vonmises.append(vessel.get_vonmises_stress())
            print('predicted vonmises is:',predicted_vonmises)
            convergence=check_convergence(predicted_vonmises,mesh_lengths,experiment_id)
            if convergence==True: 
              break  
      if k==2 :
       vessel.csv_write_row(csvwriter)
       simulated_index.append(batch_start+i)
      else:
       skipped.append(batch_start+i)
     else: 
       skipped.append(batch_start+i)
     with open ("convergence.txt","w")as fp: 
       fp.write(str(convergence_data))
vessel.csv_close_output(csvwriter)



mesh option is : [0.098 0.096 0.094 0.092 0.09  0.088 0.086 0.084 0.082 0.08  0.078 0.076
 0.074 0.072 0.07  0.068 0.066 0.064 0.062 0.06  0.058 0.056 0.054 0.052
 0.05  0.048 0.046 0.044 0.042 0.04  0.038 0.036 0.034 0.032 0.03 ]
---> depth is: 3500.0 length is: 0.2 thickness is: 0.36000000000000004 radius is: 0.6664557859122784
Something went wrong in setting value
---> depth is: 5700.0 length is: 0.8 thickness is: 0.06999999999999999 radius is: 0.35789509084878723
ml is: 0.09800000000000006
Running GMSH mesher ... 5047 nodes, 124 edges, 1666 faces, 2542 volumes
Running FEM analysis ... vonMises stress: 437.15 MPa
itr is: 1 ,k is: 2 ,ml is: 0.09800000000000006
predicted vonmises is: [437.1502971799042]
ml is: 0.09600000000000006
Running GMSH mesher ... 5019 nodes, 126 edges, 1658 faces, 2526 volumes
Running FEM analysis ... vonMises stress: 436.01 MPa
itr is: 1 ,k is: 2 ,ml is: 0.09600000000000006
predicted vonmises is: [437.1502971799042, 436.0097690356555]
ml is: 0.0940000000000000

Running FEM analysis ... vonMises stress: 173.30 MPa
itr is: 4 ,k is: 2 ,ml is: 0.09400000000000006
predicted vonmises is: [173.89350203221426, 175.4978354043069, 173.300442933006]
ml is: 0.09200000000000005
Running GMSH mesher ... 5500 nodes, 104 edges, 1382 faces, 3081 volumes
Running FEM analysis ... vonMises stress: 179.32 MPa
itr is: 4 ,k is: 2 ,ml is: 0.09200000000000005
predicted vonmises is: [173.89350203221426, 175.4978354043069, 173.300442933006, 179.32301303541422]
ml is: 0.09000000000000005
Running GMSH mesher ... 5532 nodes, 104 edges, 1382 faces, 3107 volumes
Running FEM analysis ... vonMises stress: 177.21 MPa
itr is: 4 ,k is: 2 ,ml is: 0.09000000000000005
predicted vonmises is: [173.89350203221426, 175.4978354043069, 173.300442933006, 179.32301303541422, 177.21136375989857]
ml is: 0.08800000000000005
Running GMSH mesher ... itr is: 4 ,k is: 0 ,ml is: 0.08800000000000005
ml is: 0.08600000000000005
Running GMSH mesher ... 6651 nodes, 114 edges, 1692 faces, 3709 volumes
Ru

Running FEM analysis ... vonMises stress: 75.70 MPa
itr is: 7 ,k is: 2 ,ml is: 0.08400000000000005
predicted vonmises is: [74.7496088055641, 75.10036459528571, 76.19245775648092, 76.58472527379728, 78.42167639486408, 75.7028578362197]
ml is: 0.08200000000000005
Running GMSH mesher ... 8091 nodes, 122 edges, 1648 faces, 4849 volumes
Running FEM analysis ... vonMises stress: 77.01 MPa
itr is: 7 ,k is: 2 ,ml is: 0.08200000000000005
predicted vonmises is: [74.7496088055641, 75.10036459528571, 76.19245775648092, 76.58472527379728, 78.42167639486408, 75.7028578362197, 77.01428261250389]
windowed result is: [75.10036459528571, 76.19245775648092, 76.58472527379728, 78.42167639486408, 75.7028578362197, 77.01428261250389]
self index is: 10665.0
---> depth is: 800.0 length is: 0.7000000000000001 thickness is: 0.15000000000000002 radius is: 0.4930961149102885
ml is: 0.09800000000000006
Running GMSH mesher ... 6825 nodes, 152 edges, 2258 faces, 3422 volumes
Running FEM analysis ... vonMises stress:

Running GMSH mesher ... 6465 nodes, 172 edges, 2130 faces, 3264 volumes
Running FEM analysis ... vonMises stress: 269.48 MPa
itr is: 11 ,k is: 2 ,ml is: 0.09600000000000006
predicted vonmises is: [269.48126291415815, 269.48129189319155]
ml is: 0.09400000000000006
Running GMSH mesher ... 7003 nodes, 178 edges, 2298 faces, 3550 volumes
Running FEM analysis ... vonMises stress: 272.43 MPa
itr is: 11 ,k is: 2 ,ml is: 0.09400000000000006
predicted vonmises is: [269.48126291415815, 269.48129189319155, 272.43132230975107]
ml is: 0.09200000000000005
Running GMSH mesher ... 7226 nodes, 182 edges, 2374 faces, 3659 volumes
Running FEM analysis ... vonMises stress: 276.49 MPa
itr is: 11 ,k is: 2 ,ml is: 0.09200000000000005
predicted vonmises is: [269.48126291415815, 269.48129189319155, 272.43132230975107, 276.4915725503048]
ml is: 0.09000000000000005
Running GMSH mesher ... 7742 nodes, 188 edges, 2550 faces, 3911 volumes
Running FEM analysis ... vonMises stress: 288.78 MPa
itr is: 11 ,k is: 2 ,ml 

Running GMSH mesher ... 9923 nodes, 182 edges, 2482 faces, 5568 volumes
Running FEM analysis ... vonMises stress: 214.41 MPa
itr is: 13 ,k is: 2 ,ml is: 0.08400000000000005
predicted vonmises is: [237.5063769908084, 233.28536352529704, 197.99177443941574, 202.90259787092427, 227.74688229613596, 224.66335382941298, 220.78959662155734, 214.40678184756658]
windowed result is: [197.99177443941574, 202.90259787092427, 227.74688229613596, 224.66335382941298, 220.78959662155734, 214.40678184756658]
ml is: 0.08200000000000005
Running GMSH mesher ... 10268 nodes, 184 edges, 2518 faces, 5811 volumes
Running FEM analysis ... vonMises stress: 213.94 MPa
itr is: 13 ,k is: 2 ,ml is: 0.08200000000000005
predicted vonmises is: [237.5063769908084, 233.28536352529704, 197.99177443941574, 202.90259787092427, 227.74688229613596, 224.66335382941298, 220.78959662155734, 214.40678184756658, 213.9427557151679]
windowed result is: [202.90259787092427, 227.74688229613596, 224.66335382941298, 220.78959662155734,

...

---> depth is: 4700.0 length is: 0.2 thickness is: 0.32 radius is: 0.47322771648757
ml is: 0.09800000000000006
Running GMSH mesher ... 5442 nodes, 110 edges, 1090 faces, 3265 volumes
Running FEM analysis ... vonMises stress: 112.98 MPa
itr is: 253 ,k is: 2 ,ml is: 0.09800000000000006
predicted vonmises is: [112.9795975130463]
ml is: 0.09600000000000006
Running GMSH mesher ... 5523 nodes, 112 edges, 1106 faces, 3314 volumes
Running FEM analysis ... vonMises stress: 116.62 MPa
itr is: 253 ,k is: 2 ,ml is: 0.09600000000000006
predicted vonmises is: [112.9795975130463, 116.62158105740977]
ml is: 0.09400000000000006
Running GMSH mesher ... 5953 nodes, 114 edges, 1158 faces, 3600 volumes
Running FEM analysis ... vonMises stress: 116.02 MPa
itr is: 253 ,k is: 2 ,ml is: 0.09400000000000006
predicted vonmises is: [112.9795975130463, 116.62158105740977, 116.016059283132]
ml is: 0.09200000000000005
Running GMSH mesher ... 6556 nodes, 118 edges, 1288 faces, 3952 volumes
Running FEM analysis ... vo

Running GMSH mesher ... 23466 nodes, 230 edges, 4564 faces, 14294 volumes
Running FEM analysis ... vonMises stress: 27.18 MPa
itr is: 255 ,k is: 2 ,ml is: 0.09000000000000005
predicted vonmises is: [26.28976251904151, 24.095975989472247, 26.17717951298612, 27.418502063260128, 27.18045964555603]
ml is: 0.08800000000000005
Running GMSH mesher ... 24800 nodes, 236 edges, 4808 faces, 15108 volumes
Running FEM analysis ... vonMises stress: 93.46 MPa
itr is: 255 ,k is: 2 ,ml is: 0.08800000000000005
predicted vonmises is: [26.28976251904151, 24.095975989472247, 26.17717951298612, 27.418502063260128, 27.18045964555603, 93.45899433301163]
ml is: 0.08600000000000005
Running GMSH mesher ... 25071 nodes, 238 edges, 4740 faces, 15381 volumes
Running FEM analysis ... vonMises stress: 28.92 MPa
itr is: 255 ,k is: 2 ,ml is: 0.08600000000000005
predicted vonmises is: [26.28976251904151, 24.095975989472247, 26.17717951298612, 27.418502063260128, 27.18045964555603, 93.45899433301163, 28.9209327418377]
wi

Running GMSH mesher ... 4162 nodes, 128 edges, 1380 faces, 2086 volumes
Running FEM analysis ... vonMises stress: 177.39 MPa
itr is: 257 ,k is: 2 ,ml is: 0.09200000000000005
predicted vonmises is: [173.74147790523622, 175.44834856207683, 174.2514948387531, 177.38609913268286]
ml is: 0.09000000000000005
Running GMSH mesher ... 4330 nodes, 130 edges, 1434 faces, 2173 volumes
Running FEM analysis ... vonMises stress: 180.87 MPa
itr is: 257 ,k is: 2 ,ml is: 0.09000000000000005
predicted vonmises is: [173.74147790523622, 175.44834856207683, 174.2514948387531, 177.38609913268286, 180.87494473552715]
ml is: 0.08800000000000005
Running GMSH mesher ... 4477 nodes, 132 edges, 1480 faces, 2251 volumes
Running FEM analysis ... vonMises stress: 181.41 MPa
itr is: 257 ,k is: 2 ,ml is: 0.08800000000000005
predicted vonmises is: [173.74147790523622, 175.44834856207683, 174.2514948387531, 177.38609913268286, 180.87494473552715, 181.41175123690306]
ml is: 0.08600000000000005
Running GMSH mesher ... 4530

Running GMSH mesher ... 42179 nodes, 252 edges, 6410 faces, 27068 volumes
Running FEM analysis ... vonMises stress: 36.51 MPa
itr is: 259 ,k is: 2 ,ml is: 0.07600000000000004
predicted vonmises is: [29.535846287438265, 31.40597633397674, 31.309949654383033, 29.512478537556138, 31.02112013083667, 30.736724292609644, 35.339183500934205, 32.73009433389706, 34.79400648588764, 39.75832725144759, 32.70794890700247, 36.50961435070905]
windowed result is: [35.339183500934205, 32.73009433389706, 34.79400648588764, 39.75832725144759, 32.70794890700247, 36.50961435070905]
ml is: 0.07400000000000004
Running GMSH mesher ... 45948 nodes, 260 edges, 6808 faces, 29642 volumes
Running FEM analysis ... vonMises stress: 58.02 MPa
itr is: 259 ,k is: 2 ,ml is: 0.07400000000000004
predicted vonmises is: [29.535846287438265, 31.40597633397674, 31.309949654383033, 29.512478537556138, 31.02112013083667, 30.736724292609644, 35.339183500934205, 32.73009433389706, 34.79400648588764, 39.75832725144759, 32.70794890

Running GMSH mesher ... itr is: 259 ,k is: 0 ,ml is: 0.05200000000000002
ml is: 0.05000000000000002
Running GMSH mesher ... 135360 nodes, 382 edges, 14656 faces, 91380 volumes
Running FEM analysis ... vonMises stress: 45.10 MPa
itr is: 259 ,k is: 2 ,ml is: 0.05000000000000002
predicted vonmises is: [29.535846287438265, 31.40597633397674, 31.309949654383033, 29.512478537556138, 31.02112013083667, 30.736724292609644, 35.339183500934205, 32.73009433389706, 34.79400648588764, 39.75832725144759, 32.70794890700247, 36.50961435070905, 58.01637808359549, 34.803373191827504, 36.61653546568271, 37.66567998234074, 36.787308725052185, 90.27820774505884, 36.93448612637925, 38.60701598342447, 40.75203201368899, 41.86359998607263, 43.24020125106149, 45.09812427453275]
windowed result is: [36.93448612637925, 38.60701598342447, 40.75203201368899, 41.86359998607263, 43.24020125106149, 45.09812427453275]
ml is: 0.048000000000000015
Running GMSH mesher ... 152624 nodes, 398 edges, 15930 faces, 103461 volu

In [13]:
random.sample(range(0, 1000), 10)

[573, 746, 744, 96, 364, 143, 951, 344, 37, 784]

In [26]:
itr

NameError: name 'itr' is not defined